In [20]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, LayerNormalization, TimeDistributed
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np


In [21]:
# Download Shakespeare text
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath, 'r', encoding='utf-8') as f:
    text = f.read().lower()  # Lowercase to reduce vocabulary size

# Inspect the first 100 characters
print(text[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [22]:
# Create vocabulary
chars = sorted(set(text))
max_id = len(chars)
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

print(f"Vocabulary size: {max_id}")
print(f"Unique characters: {chars}")

Vocabulary size: 39
Unique characters: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [23]:
import numpy as np

# Convert text to integers
text_as_int = np.array([char_to_idx[c] for c in text])

# Create a TensorFlow Dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [24]:
# Create sequences of length 100
seq_length = 100
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Create input-target pairs
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [25]:
# Shuffle, batch, and prefetch
dataset = dataset.shuffle(10000).batch(64, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [27]:
from tensorflow.keras.layers import GRU, Dense, Embedding, LayerNormalization, TimeDistributed

# Build the model
model = tf.keras.Sequential([
    Embedding(max_id, 64, input_shape=[None]),
    GRU(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),
    LayerNormalization(),
    GRU(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),
    LayerNormalization(),
    TimeDistributed(Dense(max_id, activation='softmax'))
])

# Compile with Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

/usr/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(dataset, epochs=50, callbacks=[reduce_lr, early_stopping])

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 30s 143ms/step - accuracy: 0.2624 - loss: 2.6717 - learning_rate: 0.0010
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 26s 148ms/step - accuracy: 0.3961 - loss: 2.0363 - learning_rate: 0.0010
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 26s 146ms/step - accuracy: 0.4367 - loss: 1.8845 - learning_rate: 0.0010
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 24s 138ms/step - accuracy: 0.4617 - loss: 1.7918 - learning_rate: 0.0010
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 25s 144ms/step - accuracy: 0.4767 - loss: 1.7319 - learning_rate: 0.0010
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 25s 142ms/step - accuracy: 0.4888 - loss: 1.6874 - learning_rate: 0.0010
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 25s 140ms/step - accuracy: 0.4976 - loss: 1.6554 - learning_rate: 0.0010
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 25s 141ms/step - accuracy: 0.5040 - loss: 1.6291 - learning_rate: 0.0010
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 23s 130ms/step - accuracy: 0.5100 - loss: 1.6067 - learn

In [29]:
def generate_text(model, char_to_idx, idx_to_char, seed_text, length=200, temperature=0.7):
    text = seed_text.lower()
    generated = []
    for _ in range(length):
        input_ids = [char_to_idx.get(c, 0) for c in text[-seq_length:]]
        input_ids = tf.expand_dims(input_ids, 0)
        preds = model(input_ids, training=False)[0, -1, :]
        preds = tf.math.log(preds) / temperature
        next_idx = tf.random.categorical(preds[tf.newaxis, :], num_samples=1)
        next_idx = int(next_idx[0, 0])
        generated.append(idx_to_char[next_idx])
        text += idx_to_char[next_idx]
    return seed_text + ''.join(generated)

In [30]:
seed = "To be or not to be"
generated_text = generate_text(model, char_to_idx, idx_to_char, seed)
print(generated_text)

To be or not to be hath can do near part.

plance:
i should be a virtuous end them to the soldier,
when i love the regroop. the proud my how the letter:
ay, i come that the sun,
and thy scarred the serves, with the wor


In [31]:
generate_text(model, char_to_idx, idx_to_char, "I am a Student")

'I am a Student to the care\nand wail have with the lady with all be fight:\nand they shall be the present of a man; it is soul,\nwhat, i were a day my volscies of your incress they revel that you do you the myself\ntha'

In [32]:
# after training finishes
model.save("shakespeare_generator_lite.h5")
